# YouTube - summary generator
Use YT videos transcript and summarize it

In [7]:
! pip install youtube-transcript-api
! pip install pytube

  Obtaining dependency information for youtube-transcript-api from https://files.pythonhosted.org/packages/33/c1/18e32c7cd693802056f385c3ee78825102566be94a811b6556f17783c743/youtube_transcript_api-0.6.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 9.8 MB/s eta 0:00:00


In [2]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [4]:
load_dotenv() # Load environment variables from .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # Get API key from environment variable
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__7fe96d9dfa664e99ad0e78c2f9302178"
os.environ["LANGCHAIN_PROJECT"] = "youtube_template"

1. Simple Video

In [38]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=pNcQ5XXMgH4", add_video_info=True)

In [39]:
result = loader.load()

In [40]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print(result[0].metadata)
print ("")
print (result)

<class 'list'>
Found video from Greg Kamradt (Data Indy) that is 668 seconds long
{'source': 'pNcQ5XXMgH4', 'title': 'LangChain 101: YouTube Transcripts + OpenAI', 'description': 'Unknown', 'view_count': 17764, 'thumbnail_url': 'https://i.ytimg.com/vi/pNcQ5XXMgH4/hqdefault.jpg?sqp=-oaymwEXCJADEOABSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLCmP9TXvB4nm22ZX7b5Tl0AagEU3A', 'publish_date': '2023-02-23 00:00:00', 'length': 668, 'author': 'Greg Kamradt (Data Indy)'}

[Document(page_content="what is going on good people again right now we have a super exciting tutorial because we are going to take YouTube transcripts and we're going to pass them to open Ai and the way that we're going to do that is via a library called Lang chain which is what this entire series is about now before we jumped into it I wanted to show a diagram again I think these diagrams are helpful but you have to let me know so just let me know in the comments here so I wanted to do an overview about what we're actually going to be w

In [41]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [43]:
# Summarize
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
video_summary = chain.run(result)

In [44]:
video_summary

' This tutorial explains how to use the Lang Chain library to take YouTube transcripts and pass them to Open AI to generate a summary. It also explains how to use the recursive character splitter to split up long transcripts into smaller chunks, and how to use the mapreduce method to generate a summary of multiple videos. Finally, it explains how to use the summarize scan to generate a summary of multiple videos.'

### 2. Long Video (use map-reduce)

    When video is too long, it will not fit into context window. Use map-reduce chain using chanks of transcript

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(result)

In [20]:
len(texts)

14

In [21]:
#print(texts)
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"what is going on good people again right now we have a super exciting tutorial because we are going to take YouTube transcripts and we're going to pass them to open Ai and the way that we're going to do that is via a library called Lang chain which is what this entire series is about now before we jumped into it I wanted to show a diagram again I think these diagrams are helpful but you have to let me know so just let me know in the comments here so I wanted to do an overview about what we're actually going to be writing out in code because I think it's a little easier to see in pictures first so the way this is going to work is we're going to have a video a YouTube video we're going to pass it we're going to pass it a URL and then what Lang chain is going to help us do is it's going to help us load this video as a document and a doc

' This tutorial explains how to use Open AI and Lang Chain to generate summaries of multiple documents, such as YouTube videos. It provides a diagram to help visualize the code and explains how to use the mapreduce method to combine documents into one concise summary. The speaker also encourages viewers to leave comments about how the videos can be improved and about their own business problems.'

### 3. Multiple Videos

In [22]:
youtube_url_list = ["https://www.youtube.com/watch?v=UfL7hqGBLAQ", "https://www.youtube.com/watch?v=9z7p28FhoEc"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()
    
    texts.extend(text_splitter.split_documents(result))

In [24]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

" Congressman Elise Stefanik of New York's 21st Congressional District is supportive of the impeachment inquiry into President Biden and is committed to helping re-elect President Trump in 2024. A search and rescue operation is underway in Morocco following a devastating earthquake that killed 2,900 people and left hundreds of thousands homeless. International search and rescue teams have been invited by the Moroccan military to the town of Amiz, and volunteers, family, friends, local charities, and NOS are helping people in the mountains. There has been criticism that other international agencies and countries have not been able to get into Morocco."

### 4. Extract topic title & Short Description

In [25]:
# LangChain basics
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import create_extraction_chain

# Vector Store and retrievals
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
#from langchain.vectorstores import Pinecone
#import pinecone

# Chat Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Supporting libraries
import os
from dotenv import load_dotenv

load_dotenv()

True

Let's start with our map prompt which will iterate over the chunks we just made

In [26]:
template="""
You are a helpful assistant that helps retrieve topics talked about in a podcast transcript
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics include:
  - Themes
  - Business Ideas
  - Interesting Stories
  - Money making businesses
  - Quick stories about people
  - Mental Frameworks
  - Stories about an industry
  - Analogies mentioned
  - Advice or words of caution
  - Pieces of news or current events
- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the podcast
- Do not respond with anything outside of the podcast. If you don't see any topics, say, 'No Topics'
- Do not respond with numbers, just bullet points
- Do not include anything about 'Marketing Against the Grain'
- Only pull topics from the transcript. Do not use the examples
- Make your titles descriptive but concise. Example: 'Shaan's Experience at Twitch' should be 'Shaan's Interesting Projects At Twitch'
- A topic should be substantial, more than just a one-off comment

% START OF EXAMPLES
 - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
 - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
 - Revenge Against The Spam Calls: A couple of businesses focused on protecting consumers: RoboCall, TrueCaller, DoNotPay, FitIt
 - Wildcard CEOs vs. Prudent CEOs: However, Munger likes to surround himself with prudent CEO's and says he would never hire Musk.
 - Chess Business: Priyav, a college student, expressed his doubts on the MFM Facebook group about his Chess training business, mychesstutor.com, making $12.5K MRR with 90 enrolled.
 - Restaurant Refiller: An MFM Facebook group member commented on how they pay AirMark $1,000/month for toilet paper and toilet cover refills for their restaurant. Shaan sees an opportunity here for anyone wanting to compete against AirMark.
 - Collecting: Shaan shared an idea to build a mobile only marketplace for a collectors' category; similar to what StockX does for premium sneakers.
% END OF EXAMPLES
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_map = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

Then we have our combine prompt which will run once over the results of the map prompt above

In [29]:
template="""
You are a helpful assistant that helps retrieve topics talked about in a podcast transcript
- You will be given a series of bullet topics of topics vound
- Your goal is to exract the topic names and brief 1-sentence description of the topic
- Deduplicate any bullet points you see
- Only pull topics from the transcript. Do not use the examples

% START OF EXAMPLES
 - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
 - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
% END OF EXAMPLES
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_combine = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

Create your LLMs and get data

In [27]:
# Creating two versions of the model so I can swap between gpt3.5 and gpt4
llm3 = ChatOpenAI(temperature=0,
                  model_name="gpt-3.5-turbo-0613",
                  request_timeout = 180
                )

llm4 = ChatOpenAI(temperature=0,
                  model_name="gpt-4-0613",
                  request_timeout = 180
                 )

In [28]:
# Use YT transcript
print (f"You have {len(texts)} docs. First doc is {llm3.get_num_tokens(texts[0].page_content)} tokens")

You have 6 docs. First doc is 582 tokens


Store YouTube transcripts in Chroma to later use for retrival

In [ ]:
from langchain.vectorstores import Chroma
import chromadb

client = chromadb.HttpClient(host="20.115.73.2", port=8000)



#### Step 1: Extract topic title and Description

In [30]:
chain = load_summarize_chain(llm4,
                             chain_type="map_reduce",
                             map_prompt=chat_prompt_map,
                             combine_prompt=chat_prompt_combine,
                             verbose=True
                            )

In [32]:
topics_found = chain.run({"input_documents": texts})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant that helps retrieve topics talked about in a podcast transcript
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics include:
  - Themes
  - Business Ideas
  - Interesting Stories
  - Money making businesses
  - Quick stories about people
  - Mental Frameworks
  - Stories about an industry
  - Analogies mentioned
  - Advice or words of caution
  - Pieces of news or current events
- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the podcast
- Do not respond with anything outside of the podcast. If you don't see any topics, say, 'No Topics'
- Do not respond with numbers, just bullet points
- Do not include anything about 'Marketing Against the Grain'
- Only pull topics from the transcript. Do not

In [33]:
print (topics_found)

- Impeachment Inquiry into President Biden: The podcast discusses the impeachment inquiry into President Biden over his alleged involvement with Hunter's business dealings.
- Biden Family's Business Dealings: The podcast reveals that House Republicans have uncovered over 20 LLCs used by the Biden family and over $20 million from China, Russia, Ukraine, and Romania going into the Biden family.
- Alleged Bribery Scheme: The podcast mentions a bribery scheme involving Burisma and the Obama administration's policy for Ukraine, according to confidential human sources.
- Importance of Impeachment Inquiry: The podcast emphasizes the importance of the impeachment inquiry as it ensures Congress's power when it comes to deposition and subpoena, and it is seen as a crucial step towards transparency.
- Potential Vote on Bribery Scheme: Speculation on whether a vote on the alleged bribery scheme would pass, with the belief that more members might support it as they see the facts.
- Donald Trump's R

Structured Data - Turn your LLM output into structured data

In [34]:
schema = {
    "properties": {
        # The title of the topic
        "topic_name": {
            "type": "string",
            "description" : "The title of the topic listed"
        },
        # The description
        "description": {
            "type": "string",
            "description" : "The description of the topic listed"
        },
        "tag": {
            "type": "string",
            "description" : "The type of content being described",
            "enum" : ['Business Models', 'Life Advice', 'Health & Wellness', 'Stories']
        }
    },
    "required": ["topic", "description"],
}

In [35]:
# Using gpt3.5 here because this is an easy extraction task and no need to jump to gpt4
chain = create_extraction_chain(schema, llm3)

In [36]:
topics_structured = chain.run(topics_found)

In [37]:
topics_structured

[{'topic_name': 'Impeachment Inquiry into President Biden',
  'description': "The podcast discusses the impeachment inquiry into President Biden over his alleged involvement with Hunter's business dealings.",
  'tag': 'Business Models'},
 {'topic_name': "Biden Family's Business Dealings",
  'description': 'The podcast reveals that House Republicans have uncovered over 20 LLCs used by the Biden family and over $20 million from China, Russia, Ukraine, and Romania going into the Biden family.',
  'tag': 'Business Models'},
 {'topic_name': 'Alleged Bribery Scheme',
  'description': "The podcast mentions a bribery scheme involving Burisma and the Obama administration's policy for Ukraine, according to confidential human sources.",
  'tag': 'Business Models'},
 {'topic_name': 'Importance of Impeachment Inquiry',
  'description': "The podcast emphasizes the importance of the impeachment inquiry as it ensures Congress's power when it comes to deposition and subpoena, and it is seen as a crucia